In [1]:
import os
import sys
import time
import sqlite3
import multiprocessing
sys.path.append('/home/dnlab/Jupyter-Bitcoin/Heuristics/')
from secret import rpc_user, rpc_password
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException

import cluster_db_query as cdq
import db_query as dq

rpc_ip = '127.0.0.1'
rpc_port = '8332'
timeout = 300

In [2]:
def is_mi_cond(in_addrs, out_addrs):
    if in_addrs == None or out_addrs == None:
        return False
    if len(out_addrs) == 1 and len(in_addrs) >= 2:
        return True
    return False

In [3]:
def get_rpc():
    return AuthServiceProxy(f'http://{rpc_user}:{rpc_password}@{rpc_ip}:{rpc_port}', timeout=timeout)

def rpc_command(height):
    while True:
        try:
            rpc_connection = get_rpc()
            block_hash = rpc_connection.getblockhash(height)
            txes = rpc_connection.getblock(block_hash)['tx']
            break
        except OSError as e:
            print("Cannot assign requested address!")
            time.sleep(3)
    return txes

In [4]:
cluster_db_path = '/home/dnlab/Jupyter-Bitcoin/Heuristics/DB/cluster_8_multi.db'
index_db_path = '/home/dnlab/Jupyter-Bitcoin/index.db'
i_conn = sqlite3.connect(index_db_path)
c_conn = sqlite3.connect(cluster_db_path)
icur = i_conn.cursor()
ccur = c_conn.cursor()

In [5]:
# 가변길이
height = 100000

In [ ]:
false_list = list()
tx_count = 0
for height in range(0, 620000):
    txes = rpc_command(height)
    #print("tx length:", len(txes))
    for tx in txes:
        
        tx_indexes = dq.get_txid(tx)
        in_addrs = dq.get_addr_txin(tx_indexes)
        out_addrs = dq.get_addr_txout(tx_indexes)
        #print(f'tx: {tx_indexes}, in_addrs:{in_addrs}, out_addrs:{out_addrs}')
        if is_mi_cond(in_addrs, out_addrs):
            tx_count += 1
            #print(f'tx: {tx_indexes}, in_addrs:{in_addrs}, out_addrs:{out_addrs}')
            #cluster_set = cdq.get_cluster_number(in_addrs)
            #if len(cluster_set) > 1:
            #    print("[TEST CASE FALSE] different cluster_number", tx_indexes, len(in_addrs), len(out_addrs), cluster_set)
            #    false_case = {'false_num':0, 'tx':tx_indexes, 'in_addrs': in_addrs, \
            #                  'out_addrs':out_addrs, 'cluster_set':cluster_set}
            #    false_list.append(false_case)
            #elif cluster_set.pop() == -1:
            #    print("[TEST CASE FALSE] not clustered tx:", tx_indexes,", in_addrs: ", len(in_addrs), len(out_addrs), ", cluster_set", '{-1}')
            #    false_case = {'false_num':1, 'tx':tx, 'in_addrs': in_addrs, \
            #                  'out_addrs':out_addrs, 'cluster_set':{-1}}
            #    print(in_addrs)
            #    false_list.append(false_case)
            #else:
            #    print("*TEST CASE PASS*")
    if height % 100000 == 0:
        print(tx_count)


0
6730
2094734


In [15]:
import csv
with open('/home/dnlab/Jupyter-Bitcoin/Heuristics/debug/20070013_single.csv', 'a') as f:
    fieldnames = ['false_num', 'tx', 'in_addrs','out_addrs','cluster_set']
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for i in false_list:
        writer.writerow(i)     

In [16]:
tx_count#216573


6730